In [3]:
import cv2
import torch
from torchvision.transforms import functional as F
from torchvision import transforms
import numpy as np
import matplotlib.pyplot as plt

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = torch.load("bestmodelv5x2_SSD320lite.pth", map_location=device)
model.eval()

C:\Users\THINH\AppData\Local\Temp\ipykernel_6372\641032312.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load("bestmodelv5x2_SSD320lite.pth", map_locatio

SSD(
  (backbone): SSDLiteFeatureExtractorMobileNet(
    (features): Sequential(
      (0): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (2): Hardswish()
        )
        (1): InvertedResidual(
          (block): Sequential(
            (0): Conv2dNormActivation(
              (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
              (1): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
              (2): ReLU(inplace=True)
            )
            (1): Conv2dNormActivation(
              (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
              (1): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
            )
          )
        )
        (2): Invert

In [5]:
category_names = ["none", "A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y", "Z"]

In [6]:
cap = cv2.VideoCapture(0)

In [7]:
if not cap.isOpened():
    print("Error: Could not open video stream.")
    exit()

# Lọc dự đoán với ngưỡng confidence
threshold = 0.7

while True:
    # Đọc một khung hình từ camera
    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to capture image")
        break

    # Chuyển đổi màu sắc từ BGR (OpenCV) sang RGB (PIL)
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Chuyển ảnh sang tensor và chuẩn hóa
    img_tensor = F.to_tensor(img).unsqueeze(0).to(device)

    # Dự đoán với mô hình
    with torch.no_grad():  # Không cần tính gradient cho quá trình dự đoán
        prediction = model(img_tensor)

    # Trích xuất các kết quả dự đoán
    boxes = prediction[0]['boxes'].cpu().numpy()
    labels = prediction[0]['labels'].cpu().numpy()
    scores = prediction[0]['scores'].cpu().numpy()

    # Lọc các kết quả dự đoán có điểm số > threshold
    filtered_boxes = boxes[scores > threshold]
    filtered_labels = labels[scores > threshold]
    filtered_scores = scores[scores > threshold]

    # Vẽ bounding boxes lên ảnh
    for box, label, score in zip(filtered_boxes, filtered_labels, filtered_scores):
        xmin, ymin, xmax, ymax = box
        if label < len(category_names):
            class_name = category_names[label]
        else:
            class_name = "Unknown"  # Nếu label không hợp lệ, gán là "Unknown"
        
        # Vẽ hộp giới hạn và tên lớp
        cv2.rectangle(frame, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (255, 0, 0), 2)  # Màu đỏ
        cv2.putText(frame, f'{class_name}: {score:.2f}', (int(xmin), int(ymin)-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    # Chuyển đổi lại BGR để hiển thị bằng OpenCV
    frame_bgr = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

    # Hiển thị video với các bounding boxes
    cv2.imshow('Object Detection', frame_bgr)

    # Thoát vòng lặp khi nhấn phím 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


In [8]:
cap.release()
cv2.destroyAllWindows()